In [15]:
import pandas as pd

expo_import = pd.read_excel("ExpoArchive_cleaned.xlsx")
purchasing_import = pd.read_excel("PurchasingTool_cleaned.xlsx")
scorecard_import = pd.read_excel('Scorecard_cleaned.xlsx')

In [16]:
expo = expo_import
purchasing = purchasing_import
scorecard = scorecard_import

In [17]:
# create return spend and return quantity columns. Zero out negatives in spend  and qty
# transaction_type is 0 = return, 1 = order

purchasing['transaction_type'] = [0 if d < 0 else 1 for d in purchasing['Spend']]

purchasing['Return_spend'] = [0 if d >= 0 else abs(d) for d in purchasing['Spend']]
purchasing['Spend'] = [d if d >= 0 else 0 for d in purchasing['Spend']]
                       
purchasing['Return_qty'] = [0 if d >= 0 else abs(d) for d in purchasing['Qty']]
purchasing['Qty'] = [d if d >= 0 else 0 for d in purchasing['Qty']]

In [18]:

# import numpy as np
# import math
 
# purchasing['Spend'] = [(np.sign(x)*math.log(abs(x) + 1)) for x in purchasing['Spend']]
# purchasing['Return_spend'] = [(np.sign(x)*math.log(abs(x) + 1)) for x in purchasing['Return_spend']]
# purchasing['Qty'] = [(np.sign(x)*math.log(abs(x) + 1)) for x in purchasing['Qty']]
# purchasing['Return_qty'] = [(np.sign(x)*math.log(abs(x) + 1)) for x in purchasing['Return_qty']]
# purchasing['StdCost'] = [(np.sign(x)*math.log(abs(x) + 1)) for x in purchasing['StdCost']]
# purchasing['PoCost'] = [(np.sign(x)*math.log(abs(x) + 1)) for x in purchasing['PoCost']]
# purchasing['InternalCostSavings'] = [(np.sign(x)*math.log(abs(x) + 1)) for x in purchasing['InternalCostSavings']]
#purchasing['PurchLeadTime'] = [(np.sign(x)*math.log(abs(x) + 1)) for x in purchasing['PurchLeadTime']]


# purchasing


In [19]:
# add single/multi source column to p_tool as 'Source'. 0 = multi_source, 1 = single_source

source = pd.DataFrame()

source['Source'] = purchasing.groupby(['Part'])['Vendor Code'].apply(lambda x: x.drop_duplicates().count())
source['Source'] = [1 if (d == 1) else 0 for d in source['Source']]
source['Part'] = source.index
source.index = range(len(source))

purchasing = purchasing.merge(source, on = ['Part'], how = 'left')

In [20]:
# transform ABC column

ABC_dict = {'A':0.7, 'B':0.5, 'C':0.3}
purchasing['ABC'] = [ABC_dict[d] for d in purchasing['ABC']]

In [21]:
# append 'Category' to expo archive

part_category = pd.DataFrame()
part_category["Part Number"] = purchasing["Part"]
part_category["Category"] = purchasing["Category"]
print len(part_category)
part_category = part_category.dropna()
print len(part_category)
part_category = part_category.drop_duplicates(subset=['Part Number'])
print len(part_category)
print part_category

236437
236437
20005
           Part Number         Category
0         L45-0043-009       Elect Comp
3         L25-9287-012       Elect Comp
4         L25-9289-024       Elect Comp
5         L26-0012-332       Elect Comp
6        10075-1365-05       Elect Comp
9             100-0010         Indirect
13            100-5029          Antenna
17           1001-0619      Fabrication
18            100-5054          Antenna
20           1002-0010         Indirect
21       10075-1365-06       Elect Comp
22          10075-1399  Accessories/OEM
24          10075-0736      Fabrication
26          0N818910-2      Fabrication
27          1000023994       Elect Comp
28          10075-1031      Fabrication
29          0N804998-8          Raw PCB
35          0N689775-4          Raw PCB
40          0N805009-1      Fabrication
41          0N816516-1          Unknown
42          0N818902-2      Fabrication
43          0N689788-2          Raw PCB
44          0N818908-3          Raw PCB
48            00AAH2

In [22]:
print len(expo)
expo = expo.merge(part_category, on = ['Part Number'], how = 'left')
expo = expo.dropna()
print len(expo)

32043
27197


In [23]:
# aggregate expo

def expo_aggregate(data, groupby):
    new_expo = pd.DataFrame()
    
    new_expo['avg_late'] = data.groupby(groupby)['Late'].mean()
    new_expo['avg_early'] = data.groupby(groupby)['Early'].mean()


    new_expo['num_late'] = data.groupby(groupby)['Late'].apply(lambda x: x[x > 0].count())
    new_expo['num_early'] = data.groupby(groupby)['Early'].apply(lambda x: x[x > 0].count())
    new_expo['num_critical'] = data.groupby(groupby)['Critical'].apply(lambda x: x[x == True].count())
    new_expo['num_transactions_expo'] = data.groupby(groupby)['Late'].count()


    new_expo['%_late'] = new_expo['num_late'] / new_expo['num_transactions_expo']
    new_expo['%_early'] = new_expo['num_early'] / new_expo['num_transactions_expo']
    new_expo['%_critical'] = new_expo['num_critical'] / new_expo['num_transactions_expo']
    #new_expo['%_singlesource'] = data.groupby(groupby)['Source'].mean()

    new_expo = new_expo.drop(['num_late'], axis=1)
    new_expo = new_expo.drop(['num_early'], axis=1)
    new_expo = new_expo.drop(['num_critical'], axis=1)


    new_expo['Vendor Code'] = new_expo.index
    new_expo.index = range(len(new_expo))
    
    new_expo['Category'] = [d[1] for d in new_expo['Vendor Code']]
    new_expo['Vendor Code'] = [d[0] for d in new_expo['Vendor Code']]
    
    return new_expo



In [24]:
new_expo = expo_aggregate(expo, ['Vendor Code', 'Category'])
new_expo

,avg_late,avg_early,num_transactions_expo,%_late,%_early,%_critical,Vendor Code,Category
0,186.500000,0.000000,2,0.500000,0.000000,0.000000,83,Fabrication
1,0.000000,0.000000,5,0.000000,0.000000,0.000000,612,Indirect
2,19.813953,1.325581,43,0.325581,0.186047,0.348837,630,Fabrication
3,0.666667,5.333333,3,0.333333,0.666667,0.000000,920,Cable/Conn/Wire
4,5.875000,0.000000,8,0.375000,0.000000,0.250000,1056,Cable/Conn/Wire
5,5.000000,6.500000,2,0.500000,0.500000,0.000000,1212,Cable/Conn/Wire
6,8.500000,3.000000,2,0.500000,0.500000,0.000000,1212,Elect Comp
7,3.833333,2.666667,6,0.333333,0.166667,0.166667,1250,Accessories/OEM
8,1.500000,0.000000,2,0.500000,0.000000,0.000000,1250,Cable/Conn/Wire
9,0.000000,4.000000,1,0.000000,1.000000,1.000000,1250,Fabrication


In [25]:
# aggregate purchasing


def purchasing_aggregate(data, groupby):
    aggregate = pd.DataFrame()
    
    aggregate['mean_log_stdcost'] = data.groupby(groupby)['StdCost'].mean()
    aggregate['median_log_stdcost'] = data.groupby(groupby)['StdCost'].median()
    aggregate['var_log_stdcost'] = data.groupby(groupby)['StdCost'].var()

    aggregate['mean_log_pocost'] = data.groupby(groupby)['PoCost'].mean()
    aggregate['median_log_pocost'] = data.groupby(groupby)['PoCost'].median()
    aggregate['var_log_pocost'] = data.groupby(groupby)['PoCost'].var()
    
    
    aggregate['mean_log_qty'] = data.groupby(groupby)['Qty'].mean()
    aggregate['median_log_qty'] = data.groupby(groupby)['Qty'].median()
    aggregate['var_log_qty'] = data.groupby(groupby)['Qty'].var()
    aggregate['sum_log_qty'] = data.groupby(groupby)['Qty'].sum()
    
    aggregate['mean_log_return_qty'] = data.groupby(groupby)['Return_qty'].mean()
    aggregate['median_log_return_qty'] = data.groupby(groupby)['Return_qty'].median()
    aggregate['var_log_return_qty'] = data.groupby(groupby)['Return_qty'].var()
    aggregate['sum_log_return_qty'] = data.groupby(groupby)['Return_qty'].sum()
    
    
    aggregate['mean_log_costsave'] = data.groupby(groupby)['InternalCostSavings'].mean()
    aggregate['median_log_costsave'] = data.groupby(groupby)['InternalCostSavings'].median()
    aggregate['var_log_costsave'] = data.groupby(groupby)['InternalCostSavings'].var()
    aggregate['sum_log_costsave'] = data.groupby(groupby)['InternalCostSavings'].sum()
    
    aggregate['mean_log_spend'] = data.groupby(groupby)['Spend'].mean()
    aggregate['median_log_spend'] = data.groupby(groupby)['Spend'].median()
    aggregate['var_log_spend'] = data.groupby(groupby)['Spend'].var()
    aggregate['sum_log_spend'] = data.groupby(groupby)['Spend'].sum()

    aggregate['mean_log_return_spend'] = data.groupby(groupby)['Return_spend'].mean()
    aggregate['median_log_return_spend'] = data.groupby(groupby)['Return_spend'].median()
    aggregate['var_log_return_spend'] = data.groupby(groupby)['Return_spend'].var()
    aggregate['sum_log_return_spend'] = data.groupby(groupby)['Return_spend'].sum()
    
    aggregate['mean_log_leadtime'] = data.groupby(groupby)['PurchLeadTime'].mean()
    aggregate['median_log_leadtime'] = data.groupby(groupby)['PurchLeadTime'].median()
    aggregate['variance_log_leadtime'] = data.groupby(groupby)['PurchLeadTime'].var()
    
    aggregate['mean_priority'] = data.groupby(groupby)['ABC'].mean()
    
    aggregate['%_singlesource'] = data.groupby(groupby)['Source'].mean()


    aggregate['num_orders_purch'] = data.groupby(groupby)['transaction_type'].apply(lambda x: x[x == 1].count())
    aggregate['num_returns_purch'] = data.groupby(groupby)['Spend'].apply(lambda x: x[x == 0].count())

    

    #aggregate['num_a_priority'] = data.groupby(groupby)['ABC'].apply(lambda x: x[x == 'A'].count())
    #aggregate['num_b_priority'] = data.groupby(groupby)['ABC'].apply(lambda x: x[x == 'B'].count())
    #aggregate['num_c_priority'] = data.groupby(groupby)['ABC'].apply(lambda x: x[x == 'C'].count())

    #aggregate['%_a_priority'] = aggregate['num_a_priority'] / aggregate['num_orders_purch']
    #aggregate['%_b_priority'] = aggregate['num_b_priority'] / aggregate['num_orders_purch']
    #aggregate['%_c_priority'] = aggregate['num_c_priority'] / aggregate['num_orders_purch']
    
    #aggregate = aggregate.drop('num_a_priority', axis = 1)
    #aggregate = aggregate.drop('num_b_priority', axis = 1)
    #aggregate = aggregate.drop('num_c_priority', axis = 1)


    aggregate['Vendor Code'] = aggregate.index
    aggregate.index = range(len(aggregate))
    
    aggregate['Category'] = [d[1] for d in aggregate['Vendor Code']]
    aggregate['Vendor Code'] = [d[0] for d in aggregate['Vendor Code']]
    
    return aggregate

          

In [26]:
new_purchasing = purchasing_aggregate(purchasing, ['Vendor Code', 'Category'])
new_purchasing

,mean_log_stdcost,median_log_stdcost,var_log_stdcost,mean_log_pocost,median_log_pocost,var_log_pocost,mean_log_qty,median_log_qty,var_log_qty,sum_log_qty,...,sum_log_return_spend,mean_log_leadtime,median_log_leadtime,variance_log_leadtime,mean_priority,%_singlesource,num_orders_purch,num_returns_purch,Vendor Code,Category
0,6.728810,6.728810,NaN,6.810000,6.81000,NaN,100.000000,100.0,NaN,100.0,...,0.00,120.000000,120.0,NaN,0.300000,1.000000,1,0,83,Elect Comp
1,7.290000,7.290000,0.000000,7.130000,7.29000,0.076800,833.333333,1000.0,8.333333e+04,2500.0,...,0.00,120.000000,120.0,0.000000,0.300000,1.000000,3,0,83,Fabrication
2,0.790000,0.790000,NaN,0.790000,0.79000,NaN,3100.000000,3100.0,NaN,3100.0,...,0.00,35.000000,35.0,NaN,0.300000,1.000000,1,0,473,Cable/Conn/Wire
3,1.371250,0.165000,3.967696,1.369167,0.16500,3.973190,782.333333,1000.0,5.184706e+05,9388.0,...,0.00,17.083333,15.0,6.628788,0.300000,1.000000,12,0,612,Indirect
4,509.442744,485.000000,3206.505661,511.207407,485.00000,3610.658154,26.081481,20.0,4.660456e+02,3521.0,...,25220.00,70.311111,70.0,13.066667,0.590370,0.933333,132,3,630,Fabrication
5,0.807333,0.820000,0.002407,1.055333,1.18000,0.055355,6986.666667,6700.0,9.049810e+06,104800.0,...,0.00,30.000000,30.0,0.000000,0.673333,1.000000,15,0,920,Cable/Conn/Wire
6,145.000000,145.000000,50.000000,145.000000,145.00000,50.000000,25.000000,25.0,0.000000e+00,50.0,...,0.00,77.000000,77.0,98.000000,0.700000,0.000000,2,0,958,OCF
7,1.200000,1.200000,0.000000,1.200000,1.20000,0.000000,100.000000,100.0,0.000000e+00,200.0,...,0.00,30.000000,30.0,0.000000,0.300000,1.000000,2,0,993,Cable/Conn/Wire
8,4.160000,4.160000,0.000000,0.319000,0.31900,0.000000,300.000000,300.0,1.800000e+05,600.0,...,191.40,196.000000,196.0,0.000000,0.500000,0.000000,1,1,1027,Elect Comp
9,38.570905,26.100000,291.281871,38.449774,26.10000,278.344987,87.454774,100.0,3.842223e+03,34807.0,...,0.00,79.427136,56.0,715.640773,0.693970,1.000000,398,0,1056,Cable/Conn/Wire


In [27]:
join = new_purchasing.merge(new_expo, on = ['Vendor Code', 'Category'], how = 'inner')
join

,mean_log_stdcost,median_log_stdcost,var_log_stdcost,mean_log_pocost,median_log_pocost,var_log_pocost,mean_log_qty,median_log_qty,var_log_qty,sum_log_qty,...,num_orders_purch,num_returns_purch,Vendor Code,Category,avg_late,avg_early,num_transactions_expo,%_late,%_early,%_critical
0,7.290000,7.290000,0.000000,7.130000,7.29000,0.076800,833.333333,1000.0,8.333333e+04,2500.0,...,3,0,83,Fabrication,186.500000,0.000000,2,0.500000,0.000000,0.000000
1,1.371250,0.165000,3.967696,1.369167,0.16500,3.973190,782.333333,1000.0,5.184706e+05,9388.0,...,12,0,612,Indirect,0.000000,0.000000,5,0.000000,0.000000,0.000000
2,509.442744,485.000000,3206.505661,511.207407,485.00000,3610.658154,26.081481,20.0,4.660456e+02,3521.0,...,132,3,630,Fabrication,19.813953,1.325581,43,0.325581,0.186047,0.348837
3,0.807333,0.820000,0.002407,1.055333,1.18000,0.055355,6986.666667,6700.0,9.049810e+06,104800.0,...,15,0,920,Cable/Conn/Wire,0.666667,5.333333,3,0.333333,0.666667,0.000000
4,38.570905,26.100000,291.281871,38.449774,26.10000,278.344987,87.454774,100.0,3.842223e+03,34807.0,...,398,0,1056,Cable/Conn/Wire,5.875000,0.000000,8,0.375000,0.000000,0.250000
5,108.428600,116.666700,438.520653,106.071429,110.00000,431.609890,4.071429,3.0,1.222527e+01,57.0,...,14,0,1212,Cable/Conn/Wire,5.000000,6.500000,2,0.500000,0.500000,0.000000
6,294.947400,294.947400,0.000000,309.000000,315.00000,162.000000,33.750000,14.0,1.549071e+03,270.0,...,8,0,1212,Elect Comp,8.500000,3.000000,2,0.500000,0.500000,0.000000
7,130.297928,39.714000,96071.798769,210.252500,40.02000,343585.137166,48.750000,26.0,3.657477e+03,585.0,...,11,1,1250,Accessories/OEM,3.833333,2.666667,6,0.333333,0.166667,0.166667
8,2.850000,2.850000,0.000000,2.826667,2.82000,0.000175,50.777778,22.0,8.378694e+03,457.0,...,9,0,1250,Cable/Conn/Wire,1.500000,0.000000,2,0.500000,0.000000,0.000000
9,486.448000,486.448000,0.000000,467.413333,452.00000,712.712533,33.333333,44.0,5.693333e+02,100.0,...,3,0,1250,Fabrication,0.000000,4.000000,1,0.000000,1.000000,1.000000


In [28]:
feat_to_log = ['mean_log_stdcost', 'median_log_stdcost', 'var_log_stdcost',
       'mean_log_pocost', 'median_log_pocost', 'var_log_pocost',
       'mean_log_qty', 'median_log_qty', 'var_log_qty', 'sum_log_qty',
       'mean_log_return_qty', 'median_log_return_qty', 'var_log_return_qty',
       'sum_log_return_qty', 'mean_log_costsave', 'median_log_costsave',
       'var_log_costsave', 'sum_log_costsave', 'mean_log_spend',
       'median_log_spend', 'var_log_spend', 'sum_log_spend',
       'mean_log_return_spend', 'median_log_return_spend',
       'var_log_return_spend', 'sum_log_return_spend', 'mean_log_leadtime',
       'median_log_leadtime', 'variance_log_leadtime']

for feat in feat_to_log:
    join[feat] = [(np.sign(x)*math.log(abs(x) + 1)) for x in join[feat]]
    
join

,mean_log_stdcost,median_log_stdcost,var_log_stdcost,mean_log_pocost,median_log_pocost,var_log_pocost,mean_log_qty,median_log_qty,var_log_qty,sum_log_qty,...,num_orders_purch,num_returns_purch,Vendor Code,Category,avg_late,avg_early,num_transactions_expo,%_late,%_early,%_critical
0,2.115050,2.115050,0.000000,2.095561,2.115050,0.073994,6.726633,6.908755,11.330616,7.824446,...,3,0,83,Fabrication,186.500000,0.000000,2,0.500000,0.000000,0.000000
1,0.863417,0.152721,1.602956,0.862538,0.152721,1.604062,6.663558,6.908755,13.158641,9.147294,...,12,0,612,Indirect,0.000000,0.000000,5,0.000000,0.000000,0.000000
2,6.235278,6.186209,8.073249,6.238730,6.186209,8.191922,3.298850,3.044522,6.146427,8.166784,...,132,3,630,Fabrication,19.813953,1.325581,43,0.325581,0.186047,0.348837
3,0.591852,0.598837,0.002404,0.720438,0.779325,0.053877,8.851902,8.810012,16.018254,11.559819,...,15,0,920,Cable/Conn/Wire,0.666667,5.333333,3,0.333333,0.666667,0.000000
4,3.678094,3.299534,5.677719,3.675028,3.299534,5.632448,4.482491,4.615121,8.254067,10.457603,...,398,0,1056,Cable/Conn/Wire,5.875000,0.000000,8,0.375000,0.000000,0.250000
5,4.695272,4.767856,6.085685,4.673496,4.709530,6.069836,1.623623,1.386294,2.582130,4.060443,...,14,0,1212,Cable/Conn/Wire,5.000000,6.500000,2,0.500000,0.500000,0.000000
6,5.690182,5.690182,0.000000,5.736572,5.755742,5.093750,3.548180,2.708050,7.346056,5.602119,...,8,0,1212,Elect Comp,8.500000,3.000000,2,0.500000,0.500000,0.000000
7,4.877469,3.706572,11.472862,5.353054,3.714060,12.747193,3.907010,3.295837,8.204802,6.373320,...,11,1,1250,Accessories/OEM,3.833333,2.666667,6,0.333333,0.166667,0.166667
8,1.348073,1.348073,0.000000,1.341994,1.340250,0.000175,3.946961,3.135494,9.033567,6.126869,...,9,0,1250,Cable/Conn/Wire,1.500000,0.000000,2,0.500000,0.000000,0.000000
9,6.189184,6.189184,0.000000,6.149351,6.115892,6.570480,3.536117,3.806662,6.346221,4.615121,...,3,0,1250,Fabrication,0.000000,4.000000,1,0.000000,1.000000,1.000000


In [29]:
join.to_excel("category_aggregate/join_category.xlsx", index = False)
new_purchasing.to_excel("category_aggregate/ptool_category.xlsx", index = False)
new_expo.to_excel("category_aggregate/expo_category.xlsx", index = False)

In [97]:
print new_purchasing.columns
print new_expo.columns

Index([u'mean_log_stdcost', u'median_log_stdcost', u'var_log_stdcost',
       u'mean_log_pocost', u'median_log_pocost', u'var_log_pocost',
       u'mean_log_qty', u'median_log_qty', u'var_log_qty', u'sum_log_qty',
       u'mean_log_return_qty', u'median_log_return_qty', u'var_log_return_qty',
       u'sum_log_return_qty', u'mean_log_costsave', u'median_log_costsave',
       u'var_log_costsave', u'sum_log_costsave', u'mean_log_spend',
       u'median_log_spend', u'var_log_spend', u'sum_log_spend',
       u'mean_log_return_spend', u'median_log_return_spend',
       u'var_log_return_spend', u'sum_log_return_spend', u'mean_log_leadtime',
       u'median_log_leadtime', u'variance_log_leadtime', u'mean_priority',
       u'%_singlesource', u'num_orders_purch', u'num_returns_purch',
       u'Vendor Code', u'Category'],
      dtype='object')
Index([u'avg_late', u'avg_early', u'num_transactions_expo', u'%_late',
       u'%_early', u'%_critical', u'Vendor Code', u'Category'],
      dtype='object'

In [4]:
# aggregate scorecard

def scorecard_aggregate(data, groupby):

    bad_periods = ['Current Fiscal Year', 'Previous Fiscal Year', 'Last 12 Complete Periods', 'Current Period']
    bad_index = []
    for i in range(len(scorecard)):
        if scorecard.iloc[i]['Period'] in bad_periods:
            bad_index = bad_index + [i]

    data = data.drop(bad_index)

    aggregate = pd.DataFrame()

    aggregate['avg_quality'] = data.groupby(groupby)['QualityScore'].mean()
    aggregate['avg_delivery'] = data.groupby(groupby)['DeliveryScore'].mean()

    aggregate['Vendor Code'] = aggregate.index
    aggregate.index = range(len(aggregate))
    
    return aggregate
    